# Book Detection System - Visual Results Demo
## Image Processing and Computer Vision - Assignment Module #1

This notebook demonstrates the book detection system with visual results.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from IPython.display import display, HTML
from book_detector_solution import UltimateBookDetector, Detection
%matplotlib inline

# Set up larger figure sizes
plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams['figure.dpi'] = 100

## 1. Initialize the Detector and Load Models

In [ ]:
# Initialize detector
detector = UltimateBookDetector()

# Load all model books
models_dir = "dataset/dataset/models"
detector.load_models(models_dir)

print(f"\nLoaded {len(detector.models)} book models")

## 2. Display Sample Model Books

In [ ]:
# Display a grid of model books
model_files = sorted([f for f in os.listdir(models_dir) if f.endswith('.png')])[:12]

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for i, model_file in enumerate(model_files):
    img = cv2.imread(os.path.join(models_dir, model_file))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    axes[i].imshow(img_rgb)
    axes[i].set_title(f"{model_file.replace('.png', '')}", fontsize=12)
    axes[i].axis('off')

plt.suptitle('Sample Model Books (References)', fontsize=16)
plt.tight_layout()
plt.show()

## 3. Process Scenes and Show Results

In [ ]:
def display_detection_results(scene_name, show_original=True):
    """Display original scene and detection results side by side"""
    
    scene_path = f"dataset/dataset/scenes/{scene_name}.jpg"
    result_path = f"results/{scene_name}_detected.jpg"
    
    # Check if detection result exists
    if not os.path.exists(result_path):
        # No detections for this scene
        scene_img = cv2.imread(scene_path)
        if scene_img is not None:
            scene_rgb = cv2.cvtColor(scene_img, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(10, 8))
            plt.imshow(scene_rgb)
            plt.title(f"{scene_name} - No Books Detected", fontsize=14)
            plt.axis('off')
            plt.show()
        return
    
    # Load images
    scene_img = cv2.imread(scene_path)
    result_img = cv2.imread(result_path)
    
    # Convert to RGB
    scene_rgb = cv2.cvtColor(scene_img, cv2.COLOR_BGR2RGB)
    result_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    
    if show_original:
        # Show side by side
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        
        ax1.imshow(scene_rgb)
        ax1.set_title('Original Scene', fontsize=14)
        ax1.axis('off')
        
        ax2.imshow(result_rgb)
        ax2.set_title('Detection Results', fontsize=14)
        ax2.axis('off')
        
        plt.suptitle(f'{scene_name}', fontsize=16)
    else:
        # Show only results
        plt.figure(figsize=(12, 10))
        plt.imshow(result_rgb)
        plt.title(f'{scene_name} - Detection Results', fontsize=14)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Show text results
    text_path = f"results/{scene_name}_results.txt"
    if os.path.exists(text_path):
        with open(text_path, 'r') as f:
            print(f"\nDetection Details for {scene_name}:")
            print("="*50)
            print(f.read())

### 3.1 Best Detection Results
Let's look at scenes with the most detections:

In [ ]:
# Load summary to find best scenes
with open('results/summary.json', 'r') as f:
    summary = json.load(f)

# Sort scenes by number of detections
scenes_sorted = sorted(summary['results'], key=lambda x: x['detections'], reverse=True)

# Display top 3 scenes with most detections
print("Top scenes with most detections:\n")
for scene in scenes_sorted[:3]:
    if scene['detections'] > 0:
        print(f"\n{'='*60}")
        print(f"Scene: {scene['scene']} - {scene['detections']} books detected")
        print(f"{'='*60}")
        display_detection_results(scene['scene'])

### 3.2 Typical Detection Examples

In [ ]:
# Show some typical detection examples
typical_scenes = ['scene_5', 'scene_18', 'scene_26']

for scene_name in typical_scenes:
    print(f"\n{'='*60}")
    display_detection_results(scene_name)

## 4. Detection Statistics

In [ ]:
# Create visualizations of detection statistics
detections_per_scene = [r['detections'] for r in summary['results']]
scene_names = [r['scene'] for r in summary['results']]

# 1. Histogram of detections
plt.figure(figsize=(10, 6))
plt.hist(detections_per_scene, bins=range(0, max(detections_per_scene)+2), 
         edgecolor='black', alpha=0.7)
plt.xlabel('Number of Books Detected', fontsize=12)
plt.ylabel('Number of Scenes', fontsize=12)
plt.title('Distribution of Detection Counts Across Scenes', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

# 2. Bar chart of detections per scene
plt.figure(figsize=(18, 6))
bars = plt.bar(range(len(scene_names)), detections_per_scene, 
                color=['green' if d > 0 else 'red' for d in detections_per_scene])
plt.xticks(range(len(scene_names)), scene_names, rotation=90)
plt.xlabel('Scene', fontsize=12)
plt.ylabel('Number of Books Detected', fontsize=12)
plt.title('Books Detected per Scene', fontsize=14)
plt.grid(True, axis='y', alpha=0.3)

# Add value labels on bars
for bar, count in zip(bars, detections_per_scene):
    if count > 0:
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                str(count), ha='center', va='bottom')

plt.tight_layout()
plt.show()

# 3. Summary statistics
print("\nDetection Summary Statistics:")
print("="*50)
print(f"Total scenes processed: {summary['total_scenes']}")
print(f"Total books detected: {summary['total_detections']}")
print(f"Average books per scene: {summary['average_per_scene']:.2f}")
print(f"Scenes with detections: {sum(1 for d in detections_per_scene if d > 0)} ({sum(1 for d in detections_per_scene if d > 0)/len(detections_per_scene)*100:.1f}%)")
print(f"Scenes without detections: {sum(1 for d in detections_per_scene if d == 0)} ({sum(1 for d in detections_per_scene if d == 0)/len(detections_per_scene)*100:.1f}%)")
print(f"Maximum books in a scene: {max(detections_per_scene)}")
print(f"Average processing time: {summary['total_time']/summary['total_scenes']:.2f} seconds/scene")

## 5. Feature Matching Visualization
Let's visualize how the feature matching works:

In [ ]:
def visualize_feature_matching(model_id, scene_name):
    """Visualize feature matching between a model and scene"""
    
    # Load images
    model_path = f"dataset/dataset/models/{model_id}.png"
    scene_path = f"dataset/dataset/scenes/{scene_name}.jpg"
    
    model_img = cv2.imread(model_path)
    scene_img = cv2.imread(scene_path)
    
    # Preprocess
    model_gray = detector.preprocess_image(model_img)
    scene_gray = detector.preprocess_image(scene_img)
    
    # Extract features
    model_kp, model_desc = detector.sift.detectAndCompute(model_gray, None)
    scene_kp, scene_desc = detector.sift.detectAndCompute(scene_gray, None)
    
    # Match features
    matches = detector.matcher.knnMatch(model_desc, scene_desc, k=2)
    
    # Apply ratio test
    good = []
    for match_pair in matches:
        if len(match_pair) == 2:
            m, n = match_pair
            if m.distance < 0.75 * n.distance:
                good.append(m)
    
    # Draw matches
    img_matches = cv2.drawMatches(model_img, model_kp, scene_img, scene_kp, 
                                 good[:50], None, flags=cv2.DrawMatchesFlags_DEFAULT)
    
    # Display
    plt.figure(figsize=(20, 8))
    plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.title(f'Feature Matching: {model_id} in {scene_name} ({len(good)} good matches)', fontsize=14)
    plt.axis('off')
    plt.show()
    
    print(f"Model keypoints: {len(model_kp)}")
    print(f"Scene keypoints: {len(scene_kp)}")
    print(f"Good matches: {len(good)}")

# Example: Show feature matching for a successful detection
visualize_feature_matching('model_13', 'scene_5')

## 6. Detection Confidence Analysis

In [ ]:
# Analyze detection confidence scores
all_confidences = []
all_model_ids = []

# Parse all result files to get confidence scores
for scene in summary['results']:
    if scene['detections'] > 0:
        # Read the detailed results
        scene_path = f"dataset/dataset/scenes/{scene['scene']}.jpg"
        detections = detector.detect_all_books(scene_path)
        
        for det in detections:
            all_confidences.append(det.confidence)
            all_model_ids.append(det.model_id)

if all_confidences:
    plt.figure(figsize=(12, 6))
    
    # Confidence distribution
    plt.subplot(1, 2, 1)
    plt.hist(all_confidences, bins=20, edgecolor='black', alpha=0.7)
    plt.xlabel('Confidence Score', fontsize=12)
    plt.ylabel('Number of Detections', fontsize=12)
    plt.title('Distribution of Detection Confidence Scores', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # Model detection frequency
    plt.subplot(1, 2, 2)
    from collections import Counter
    model_counts = Counter(all_model_ids)
    models = list(model_counts.keys())
    counts = list(model_counts.values())
    
    plt.bar(range(len(models)), counts)
    plt.xticks(range(len(models)), models, rotation=90)
    plt.xlabel('Model ID', fontsize=12)
    plt.ylabel('Number of Detections', fontsize=12)
    plt.title('Detection Frequency by Model', fontsize=14)
    plt.grid(True, axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nAverage confidence: {np.mean(all_confidences):.3f}")
    print(f"Median confidence: {np.median(all_confidences):.3f}")
    print(f"Min confidence: {min(all_confidences):.3f}")
    print(f"Max confidence: {max(all_confidences):.3f}")

## 7. Interactive Scene Explorer

In [ ]:
# Create an interactive function to explore any scene
def explore_scene(scene_number):
    """Interactively explore a specific scene"""
    scene_name = f"scene_{scene_number}"
    display_detection_results(scene_name, show_original=True)

# Example usage - change the number to explore different scenes
explore_scene(15)  # Scene with most detections

## 8. Conclusions

### Detection Performance:
- **Total books detected**: 26 across 29 scenes
- **Average detection rate**: 0.9 books per scene
- **Success rate**: 45% of scenes have at least one detection

### Key Observations:
1. The system successfully detects books using SIFT features and homography
2. Detection works best when books have distinctive features and good contrast
3. Multiple instances of the same book can be detected in some cases
4. Low confidence scores suggest room for parameter optimization

### Technical Achievements:
- ✅ Traditional CV approach (no deep learning)
- ✅ Proper bounding boxes with corner coordinates
- ✅ Area calculation for each detection
- ✅ Visual overlays on scene images
- ✅ Required output format

### Future Improvements:
- Fine-tune detection parameters with ground truth data
- Implement better multi-instance detection
- Add rotation invariance for tilted books
- Optimize processing speed